# Variational AutoEncoder (MLP) Tensorflow
Train a variational autoencoder with MNIST dataset

#### References:
* http://kvfrans.com/variational-autoencoders-explained/
* https://github.com/kvfrans/variational-autoencoder
* https://github.com/int8/VAE_tensorflow
* http://int8.io/variational-autoencoder-in-tensorflow/
* http://blog.fastforwardlabs.com/2016/08/22/under-the-hood-of-the-variational-autoencoder-in.html
* http://blog.fastforwardlabs.com/2016/08/12/introducing-variational-autoencoders-in-prose-and.html
* https://blog.keras.io/building-autoencoders-in-keras.html
* https://jaan.io/what-is-variational-autoencoder-vae-tutorial/
* https://arxiv.org/pdf/1606.05908.pdf
* https://arxiv.org/pdf/1312.6114.pdf
* http://wiseodd.github.io/techblog/2016/12/10/variational-autoencoder/

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(0)
#os.rmdir('/tmp/vae_cnn')
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
from models import VAE_CNN
model = VAE_CNN()
model_in = model.input
model_out = model.output
model_out_flat = model.output_flat
z_mean = model.z_mean
z_stddev = model.z_stddev

### Define loss

In [3]:
with tf.name_scope("VAE_LOSS"):
    # Binary cross entropy
    generation_loss = -tf.reduce_sum(model_in * tf.log(1e-8 + model_out_flat) + (1-model_in) * tf.log(1e-8 + 1 - model_out_flat),1)
    # KL Loss
    latent_loss = 0.5 * tf.reduce_sum(tf.square(z_mean) + tf.square(z_stddev) - tf.log(tf.square(z_stddev)) - 1,1)
    loss = tf.reduce_mean(generation_loss + latent_loss)

# Solver configuration
with tf.name_scope("Solver"):
    train_step = tf.train.AdamOptimizer(0.0001).minimize(loss)

### Build Graph

In [4]:
init = tf.global_variables_initializer()

# Avoid allocating the whole memory
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

#sess = tf.Session()
sess.run(init)

### Add some tensors to observe on Tensorboard

In [5]:
tf.summary.image("input_image", model.image_in, 4)
tf.summary.image("output_image", model_out, 4)
tf.summary.scalar("loss", loss)

merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter("/tmp/vae_cnn/1")
writer.add_graph(sess.graph)

### Train

In [6]:
num_epoch = 30
batch_size = 100
for epoch in range(num_epoch):
    for i in range(int(mnist.train.num_examples / batch_size)):
        # Get batch of 50 images
        batch = mnist.train.next_batch(50)

        # Dump summary
        if i % 5000 == 0:
            s = sess.run(merged_summary, feed_dict={model_in:batch[0]})
            writer.add_summary(s,i)    


        # Train actually here (Also get loss value)    
        _, val_loss = sess.run((train_step, loss), feed_dict={model_in:batch[0]})

        #if i % 5000:
            #print('Generation loss: %d, latent loss: %d' % (np.mean(gen_loss), np.mean(lat_loss)))
    print('Epoch: %d/%d loss:%d' % (epoch, num_epoch, val_loss))

Epoch: 0/30 loss:226
Epoch: 1/30 loss:174
Epoch: 2/30 loss:152
Epoch: 3/30 loss:138
Epoch: 4/30 loss:133
Epoch: 5/30 loss:121
Epoch: 6/30 loss:120
Epoch: 7/30 loss:124
Epoch: 8/30 loss:109
Epoch: 9/30 loss:115
Epoch: 10/30 loss:113
Epoch: 11/30 loss:115
Epoch: 12/30 loss:113
Epoch: 13/30 loss:120
Epoch: 14/30 loss:110
Epoch: 15/30 loss:111
Epoch: 16/30 loss:113
Epoch: 17/30 loss:101
Epoch: 18/30 loss:110
Epoch: 19/30 loss:113
Epoch: 20/30 loss:111
Epoch: 21/30 loss:112
Epoch: 22/30 loss:109
Epoch: 23/30 loss:114
Epoch: 24/30 loss:104
Epoch: 25/30 loss:111
Epoch: 26/30 loss:109
Epoch: 27/30 loss:109
Epoch: 28/30 loss:110
Epoch: 29/30 loss:108
